In [9]:
#Import Dependencies
%matplotlib inline
import os
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import json
from pprint import pprint
from datetime import datetime
from datetime import timedelta 
import time
import matplotlib.pyplot as plt

In [10]:
!which chromedriver

/usr/local/bin/chromedriver


In [11]:
#Variables
start_date = "2008-01-01"
initial_date = datetime(2008, 1, 1)
end_date = (datetime.today() - timedelta(days = 3)).date()
print(str(initial_date.date()))

2008-01-01


In [12]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path)

In [13]:
#Banxico URL
url = 'https://www.banxico.org.mx/publicaciones-y-prensa/anuncios-de-las-decisiones-de-politica-monetaria/anuncios-politica-monetaria-t.html'
browser.visit(url)
api_url = "https://www.banxico.org.mx/SieAPIRest/service/v1/series/"
banxico_key = "85c8edeab72128102cdad764f0a2d4532c336d22690a51104b500fb6f077ed4c"

In [14]:
html = browser.html
soup = bs(html, 'html.parser')

In [15]:
links = []
for link in soup.find_all('a'):
    if ('/publicaciones-y-prensa/anuncios-de-las-decisiones-de-politica-monetaria') in link.get('href'):
        partial_link = link.get('href')
        links.append('http://www.banxico.org.mx' + partial_link)
links = links[1:]
#links

In [16]:
dates = []
for date in soup.find_all("td"):
    data = date.get_text()
    dates.append(data)
del dates[1::2]

In [17]:
import re
dates2 = []
for index in range(len(dates)):
    dates2.append(re.sub("\s", "", dates[index])) 


In [18]:
dates3 = []
for index in range(len(dates)):
    dates3.append(re.sub("[\/]", "", dates2[index])) 
#print(dates3)

In [19]:
name_counter = 0
for i in links:
    r = requests.get(i)
    out_file = open(f'{dates3[name_counter]}.pdf', 'wb')
    out_file.write(r.content)
    name_counter += 1

In [20]:
#Guillermo 
#pdf to text

In [21]:
final_dates = []
for date in dates2:
    dates = datetime.strptime(date, "%d/%m/%y")
    if dates > initial_date:
        final_dates.append(datetime.strftime(dates, "%Y-%m-%d"))
final_dates.append(datetime.strftime(end_date, "%Y-%m-%d"))

In [22]:
#MXN Lvl
Series = "SF43788"

fxt = []
for fecha in final_dates:
    url_consulta = [f"{api_url}{Series}/datos/{fecha}/{fecha}?token={banxico_key}"]
    for query in url_consulta: 
        data = requests.get(query).json()
        fxt.append(data["bmx"]["series"][0]["datos"][0]["dato"])
time.sleep(180) #tiempo del API

In [23]:
#Tasa Obejtivo
Series = "SF43878"

tasa = []
for fecha in final_dates:
    try:
        url_consulta = [f"{api_url}{Series}/datos/{fecha}/{fecha}?token={banxico_key}"]
        for query in url_consulta: 
            data = requests.get(query).json()
            tasa.append(data["bmx"]["series"][0]["datos"][0]["dato"])
    except:
        tasa.append(0)
time.sleep(180) #tiempo del API   

In [24]:
#Inflacion
Series = "SP30578"

#Total Data
total_query_url = f"{api_url}{Series}/datos/{start_date}/{end_date}?token={banxico_key}"
total_data = requests.get(total_query_url).json()
#print(total_data)
data_total = total_data['bmx']['series'][0]['datos']
data_total

[{'fecha': '01/01/2008', 'dato': '3.70000000'},
 {'fecha': '01/02/2008', 'dato': '3.72000000'},
 {'fecha': '01/03/2008', 'dato': '4.25000000'},
 {'fecha': '01/04/2008', 'dato': '4.55000000'},
 {'fecha': '01/05/2008', 'dato': '4.95000000'},
 {'fecha': '01/06/2008', 'dato': '5.26000000'},
 {'fecha': '01/07/2008', 'dato': '5.39000000'},
 {'fecha': '01/08/2008', 'dato': '5.57000000'},
 {'fecha': '01/09/2008', 'dato': '5.47000000'},
 {'fecha': '01/10/2008', 'dato': '5.78000000'},
 {'fecha': '01/11/2008', 'dato': '6.23000000'},
 {'fecha': '01/12/2008', 'dato': '6.53000000'},
 {'fecha': '01/01/2009', 'dato': '6.28000000'},
 {'fecha': '01/02/2009', 'dato': '6.20000000'},
 {'fecha': '01/03/2009', 'dato': '6.04000000'},
 {'fecha': '01/04/2009', 'dato': '6.17000000'},
 {'fecha': '01/05/2009', 'dato': '5.98000000'},
 {'fecha': '01/06/2009', 'dato': '5.74000000'},
 {'fecha': '01/07/2009', 'dato': '5.44000000'},
 {'fecha': '01/08/2009', 'dato': '5.08000000'},
 {'fecha': '01/09/2009', 'dato': '4.8900

In [58]:
#Match fechas
data_total_df = pd.DataFrame(data_total)
data_total_df['fecha'] = pd.to_datetime(data_total_df['fecha'], format='%d/%m/%Y')
data_total_df['key'] = pd.to_datetime(data_total_df['fecha']).dt.to_period('M')
data_total_df

,fecha,dato,key
0,2008-01-01,3.70000000,2008-01
1,2008-02-01,3.72000000,2008-02
2,2008-03-01,4.25000000,2008-03
3,2008-04-01,4.55000000,2008-04
4,2008-05-01,4.95000000,2008-05
...,...,...,...
143,2019-12-01,2.83000000,2019-12
144,2020-01-01,3.24000000,2020-01
145,2020-02-01,3.70000000,2020-02
146,2020-03-01,3.25000000,2020-03


In [59]:
pre_banxico_dict = {"MXN": fxt,  "Tasa": tasa}
pre_banxico_df = pd.DataFrame(pre_banxico_dict)
pre_banxico_df['Fecha'] = final_dates
pre_banxico_df['key'] = pd.to_datetime(pre_banxico_df['Fecha']).dt.to_period('M')
pre_banxico_df = pre_banxico_df.sort_values('Fecha')
pre_banxico_df = pre_banxico_df[pre_banxico_df.Tasa != 0]
pre_banxico_df = pre_banxico_df.reset_index(drop = True)
pre_banxico_df

,MXN,Tasa,Fecha,key
0,12.9370,5.0650,2009-11-27,2009-11
1,12.7040,5.0825,2010-01-15,2010-01
2,12.8085,5.0750,2010-02-19,2010-02
3,12.5685,5.0300,2010-03-19,2010-03
4,12.2810,5.0200,2010-04-16,2010-04
...,...,...,...,...
84,18.9480,7.5575,2019-12-19,2019-12
85,18.6190,7.3550,2020-02-13,2020-02
86,24.2160,7.0425,2020-03-20,2020-03
87,24.3030,6.5015,2020-04-21,2020-04


In [60]:
banxico_merge = data_total_df.merge(pre_banxico_df, on='key')
banxico_merge['Tasa'] = banxico_merge['Tasa'].astype(float)
banxico_merge['Dif'] = 0
banxico_merge.tail()


,fecha,dato,key,MXN,Tasa,Fecha,Dif
83,2019-11-01,2.97000000,2019-11,19.3760,7.7287,2019-11-14,0
84,2019-12-01,2.83000000,2019-12,18.9480,7.5575,2019-12-19,0
85,2020-02-01,3.70000000,2020-02,18.6190,7.3550,2020-02-13,0
86,2020-03-01,3.25000000,2020-03,24.2160,7.0425,2020-03-20,0
87,2020-04-01,2.15000000,2020-04,24.3030,6.5015,2020-04-21,0


In [61]:
#Fix Array
banxico_merge.rename(columns = {'dato':'Inflacion'}, inplace = True) 
banxico_merge = banxico_merge.reset_index(drop = True)
banxico_merge = banxico_merge.drop(['fecha', 'key'], axis = 1) 
banxico_merge['Tasa'] = banxico_merge['Tasa'].astype(float)
banxico_merge = banxico_merge[['Fecha', 'MXN', 'Inflacion', 'Tasa', 'Dif']]
banxico_merge

,Fecha,MXN,Inflacion,Tasa,Dif
0,2009-11-27,12.9370,3.86000000,5.0650,0
1,2010-01-15,12.7040,4.46000000,5.0825,0
2,2010-02-19,12.8085,4.83000000,5.0750,0
3,2010-03-19,12.5685,4.97000000,5.0300,0
4,2010-04-16,12.2810,4.27000000,5.0200,0
...,...,...,...,...,...
83,2019-11-14,19.3760,2.97000000,7.7287,0
84,2019-12-19,18.9480,2.83000000,7.5575,0
85,2020-02-13,18.6190,3.70000000,7.3550,0
86,2020-03-20,24.2160,3.25000000,7.0425,0


In [62]:
for i in range(1, len(banxico_merge) -1 ):
    banxico_merge.loc[i, 'Dif'] = 25 * round((100 * (banxico_merge.loc[i, 'Tasa'] - banxico_merge.loc[i-1, 'Tasa']))/25)

In [63]:
banxico_merge['key'] = pd.to_datetime(banxico_merge['Fecha']).dt.to_period('M')
banxico_merge

,Fecha,MXN,Inflacion,Tasa,Dif,key
0,2009-11-27,12.9370,3.86000000,5.0650,0.0,2009-11
1,2010-01-15,12.7040,4.46000000,5.0825,0.0,2010-01
2,2010-02-19,12.8085,4.83000000,5.0750,-0.0,2010-02
3,2010-03-19,12.5685,4.97000000,5.0300,-0.0,2010-03
4,2010-04-16,12.2810,4.27000000,5.0200,-0.0,2010-04
...,...,...,...,...,...,...
83,2019-11-14,19.3760,2.97000000,7.7287,-25.0,2019-11
84,2019-12-19,18.9480,2.83000000,7.5575,-25.0,2019-12
85,2020-02-13,18.6190,3.70000000,7.3550,-25.0,2020-02
86,2020-03-20,24.2160,3.25000000,7.0425,-25.0,2020-03


In [64]:
#import frecuencias_banxico.csv
frecuencias_df = pd.read_csv('frecuencias_banxico.csv')
frecuencias_df.rename(columns={'Unnamed: 0':'name'}, inplace=True)
frecuencias_df['key'] = frecuencias_df['name'].str.replace('count',"")
frecuencias_df['key'] = pd.to_datetime(frecuencias_df['key'].astype(str), format='%d%m%y')
frecuencias_df['key'] = pd.to_datetime(frecuencias_df['key']).dt.to_period('M')
frecuencias_df

,name,apreciacion,crecimiento,holgura,inflacion,mantener,moneda,volatilidad,depreciacion,deterioro,lasitud,disminuir,estable,aumentar,index,key
0,020818count,2.0,2.0,3.0,13.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-9.0,2018-08
1,021211count,0.0,6.0,1.0,11.0,2.0,1.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,-8.0,2011-12
2,040216count,2.0,10.0,2.0,16.0,3.0,3.0,2.0,5.0,3.0,0.0,0.0,0.0,0.0,-15.0,2016-02
3,040311count,0.0,3.0,0.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,2011-03
4,040615count,1.0,3.0,3.0,16.0,2.0,3.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,-18.0,2015-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,300616count,0.0,8.0,1.0,14.0,1.0,4.0,1.0,5.0,0.0,0.0,0.0,0.0,1.0,-16.0,2016-06
106,300715count,2.0,2.0,3.0,17.0,4.0,2.0,3.0,5.0,1.0,0.0,0.0,0.0,0.0,-21.0,2015-07
107,301112count,1.0,6.0,0.0,12.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,2012-11
108,310114count,0.0,5.0,3.0,17.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0,2014-01


In [65]:
banxico_merge = banxico_merge.merge(frecuencias_df, on='key')
banxico_merge

,Fecha,MXN,Inflacion,Tasa,Dif,key,name,apreciacion,crecimiento,holgura,...,mantener,moneda,volatilidad,depreciacion,deterioro,lasitud,disminuir,estable,aumentar,index
0,2009-11-27,12.9370,3.86000000,5.0650,0.0,2009-11,271109count,1.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0
1,2010-01-15,12.7040,4.46000000,5.0825,0.0,2010-01,150110count,0.0,2.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0
2,2010-02-19,12.8085,4.83000000,5.0750,-0.0,2010-02,190210count,0.0,2.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0
3,2010-03-19,12.5685,4.97000000,5.0300,-0.0,2010-03,190310count,0.0,1.0,1.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.0
4,2010-04-16,12.2810,4.27000000,5.0200,-0.0,2010-04,160410count,0.0,2.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2019-11-14,19.3760,2.97000000,7.7287,-25.0,2019-11,141119count,0.0,4.0,5.0,...,0.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,-5.0
86,2019-12-19,18.9480,2.83000000,7.5575,-25.0,2019-12,191219count,0.0,4.0,6.0,...,0.0,2.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,-3.0
87,2020-02-13,18.6190,3.70000000,7.3550,-25.0,2020-02,130220count,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,-1.0
88,2020-03-20,24.2160,3.25000000,7.0425,-25.0,2020-03,200320count,0.0,2.0,1.0,...,0.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,-4.0


In [66]:
# Assign X (data) and y (target)
X = banxico_merge.drop(["Dif", "Tasa", "Fecha"], axis=1)
y = banxico_merge["Dif"]
print(X.shape, y.shape)

(90, 18) (90,)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

In [ ]:
 classifier.fit(X_train, y_train)

In [ ]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

In [ ]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:101]}")
print(f"First 10 Actual labels: {y_test[:101].tolist()}")

In [ ]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)